In [1]:
library(readr)
library(readxl)
library(seqinr)
# script shared from David Bradley

In [2]:
fasta <- read.fasta('HRR25.fasta',seqtype='AA')
seq <- getSequence(fasta)

In [3]:
names(fasta)

[1] "S.cerevisiae_YPL204W_DupA"      "Z.rouxii_ZYRO0G08866g_Anc"     
 [3] "T.delbrueckii_TDEL0E05230_Anc"  "N.castellii_NCAS0A01090_DupA"  
 [5] "N.castellii_NCAS0D03820_DupB"   "V.polyspora_Kpol_1002.53_DupA" 
 [7] "V.polyspora_Kpol_1033.51_DupB"  "T.phaffii_TPHA0E02830_DupA"    
 [9] "T.phaffii_TPHA0J02370_DupB"     "K.naganishii_KNAG0F01980_DupB" 
[11] "K.naganishii_KNAG0D01130_DupA"  "K.africana_KAFR0C02290_DupB"   
[13] "K.africana_KAFR0K00320_DupA"    "N.dairenensis_NDAI0I01070_DupB"
[15] "N.dairenensis_NDAI0F01640_DupA" "T.blattae_TBLA0B05310_DupB"    
[17] "T.blattae_TBLA0A02040_DupA"     "S.bayanus"                     
[19] "S.kudriavzevii_Skud_16.75_DupA" "S.mikatae_Smik_6.409_DupA"     
[21] "C.glabrata_CAGL0H03553g_DupB"

In [4]:
pwm_files <- list.files(path='./pwm_dir/')

# Read through each PWM file to generate a list of PWMs

pwm_list <- NULL
for (i in 1:length(pwm_files)) {
  
  path <- paste('./pwm_dir/',pwm_files[i],sep='')
  
  pwm <- read.table(path)
  rownames(pwm) <- pwm[,1]
  colnames(pwm) <- NULL
  pwm <- pwm[,-1]
  
  pwm_list <- c(pwm_list, list(pwm))
  
}

names(pwm_list) <- pwm_files


In [5]:
names(pwm_list)

[1] "P11710"                 "P15891"                 "P29366-1"              
 [4] "P29366-2_PXXP"          "P32790-1_classI"        "P32790-2_classII"      
 [7] "P32790-3"               "P32793"                 "P36006"                
[10] "P38041"                 "P38753"                 "P38822-1"              
[13] "P38822-2"               "P39743_ClassI"          "P39743_ClassII"        
[16] "P39969"                 "P40073"                 "P43603"                
[19] "P47068_classIIcombined" "P53281_classI"          "P53281_classII"        
[22] "P80667_classIIA"        "P80667_classIIB"        "Q04439"                
[25] "Q05080"                 "Q06449_classI"          "Q06449_classII"        
[28] "Q07533_classI"          "Q07533_classII"         "Q12163_PXXP"

In [6]:


# Now, the PWM IDs need to be mapped to gene names

pwm_ids <- rapply(strsplit(rapply(strsplit(names(pwm_list),split='_'), function(x) x[1]),split='-'), function(x) x[1])

# Again, I will write out a file and then retrieve the gene names using the Uniprot ID mapper

#write.table(pwm_ids,file='sh3_pwm_accessions_uniprot.txt',col.names=FALSE, row.names=FALSE, quote=FALSE)

# Read in mapping table and process the strings to generate new names for the PWMs

yeast_sh3_map <- read.table('./yeast_sh3_accession_to_GN.txt',header=TRUE,stringsAsFactors = FALSE)
id_match <- yeast_sh3_map[match(pwm_ids,yeast_sh3_map[,1]),2]
pwm_names <- paste(id_match,substr(names(pwm_list),7,rapply(strsplit(names(pwm_list),split=''), function(x) length(x))),sep='')

names(pwm_list) <- pwm_names


### Here is the code for the MSS function that I use to score potential SH3 ligands


In [160]:


### Here is the code for the MSS function that I use to score potential SH3 ligands


mss_score <- function(psite,pwm) {
  
  psite <- unlist(strsplit(psite,split=''))
  pwm <- as.matrix(pwm)
    
  match.ic = apply(pwm, 2, function(col) sum(col * logb(20 * col), na.rm=T))
  
  
  c=0
  min=0
  max = 0
  
  # Ignore the central residue
  
  for (i in 1:ncol(pwm)) {
    
    aa <- psite[i]  
    
    # For missing amino acids, we score the '_' position
    # the same as we would for a minimum frequency AA
    
    if (aa == '_') {
      current <- min(pwm[,i])*match.ic[i]
    } else {
      current <- pwm[rownames(pwm) %in% psite[i],i]*match.ic[i]
    }
    
    minimum <- min(pwm[,i])*match.ic[i]
    maximum <- max(pwm[,i])*match.ic[i]
    
    c = c+current
    min = min+minimum
    max = max+maximum
    
  }
  
  mss = (c-min)/(max-min)
  
  return(unname(mss))
  
}




In [161]:
### Randomisation

### Randomly sample from the proteome

AAs <- c('A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y')

sc_proteome <- read.fasta('yeast_proteome_uniprot.fasta', seqtype='AA')
sc_aa_tab <- table(unlist(getSequence(sc_proteome)))
sc_aa_tab <- sc_aa_tab[match(AAs,names(sc_aa_tab))]
sc_aa_freq <- sc_aa_tab/sum(sc_aa_tab)



In [162]:
###### Randomisation

target_seqs <- NULL

# The whole proteome
target_seqs <- seq

max_mss_vec <- NULL

for (j in 1:length(target_seqs)) {
  
  #print(j)
  
  target_seq <- target_seqs[[j]]
  
  # Now use a completely randomised sequences of the same length
  
  target_seq <- sample(AAs, prob = sc_aa_freq, size = length(target_seq), replace=TRUE)
  
  start_site_vec <- NULL
  end_site_vec <- NULL
  
  # Generate the start and end sites
  
  for (k in 1:ncol(pwm)) {
    
    start_site <- paste(paste(rep('_',ncol(pwm)-k),collapse=''),paste(target_seq[1:k],collapse=''),sep='')
    start_site_vec <- c(start_site_vec, start_site)
    
    end_site <- paste(paste(target_seq[(length(target_seq)-k+1):length(target_seq)],collapse=''),paste(rep('_',ncol(pwm)-k),collapse=''),sep='')
    end_site_vec <- c(end_site_vec, end_site)  
    
  }
  
  # Now extract all possile binding sites from the sequence
  
  middle_site_vec <- NULL
  
  for (k in 2:(length(target_seq)-k)) {
    
    middle_site <- paste(target_seq[k:(k+ncol(pwm)-1)],collapse='')
    middle_site_vec <- c(middle_site_vec, middle_site)
    
  }
  
  candidate_sites <- NULL
  candidate_sites <- c(start_site_vec,middle_site_vec,rev(end_site_vec))
  
  # Iterate through all of the candidate sites and score with the MSS
  
  mss_vec <- NULL
  
  for (k in 1:length(candidate_sites)) {
    
    mss <- mss_score(candidate_sites[k],pwm)
    mss_vec <- c(mss_vec,mss)
    
  }
  
  max_mss <- round(as.numeric(max(mss_vec)),3)
  max_mss_vec <- c(max_mss_vec, max_mss)
}

# Use a lenient 95% threshold (usually about 0.7)

thresh <- quantile(max_mss_vec,probs=seq(0,1,0.01))[96]

rand_max_mss_vec <- max_mss_vec

In [163]:
thresh

95% 
0.64

In [11]:
# iterte through proteins
 # iterate through pwms
AAs <- c('A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y')

target_seqs <- NULL

# The whole proteome
target_seqs <- seq

max_mss_vec <- NULL
top_site_vec <- NULL
pos_vec <- NULL
name_vec <- NULL
id_vec <- NULL


In [12]:
target_seqs <- NULL


target_seqs <- seq

max_mss_vec <- NULL
top_site_vec <- NULL
pos_vec <- NULL
name_vec <- NULL
id_vec <- NULL

SH3_PWM_df <- data.frame(Gene_name=character(),Start_End=character(),PWM_name=character(),PWM_hits=character(),MSS=numeric()  )

for (z in 1:length(pwm_list)) {    

  pwm <- pwm_list[[z]]
  id <- names(pwm_list)[z]
  #print(id)
    
  ###### Proteome randomisation
  
  
  
  
  target_seqs <- NULL
  
  # The whole proteome
  target_seqs <- seq
  
  max_mss_vec <- NULL
  
  # Run it twice so that there wil be around 12,000
  # random values in the null idstribution
  
  for (l in 1:2) {
  
  for (j in 1:length(target_seqs)) {
    
    #print(j)
    
    target_seq <- target_seqs[[j]]
    
    # Now use a completely randomised sequences of the same length
    
    target_seq <- sample(AAs, prob = sc_aa_freq, size = length(target_seq), replace=TRUE)
    
    start_site_vec <- NULL
    end_site_vec <- NULL
    
    # Generate the start and end sites
    
    for (k in 1:ncol(pwm)) {
      
      start_site <- paste(paste(rep('_',ncol(pwm)-k),collapse=''),paste(target_seq[1:k],collapse=''),sep='')
      start_site_vec <- c(start_site_vec, start_site)
      
      end_site <- paste(paste(target_seq[(length(target_seq)-k+1):length(target_seq)],collapse=''),paste(rep('_',ncol(pwm)-k),collapse=''),sep='')
      end_site_vec <- c(end_site_vec, end_site)  
    }
    
    # Now extract all possile binding sites from the sequence
    middle_site_vec <- NULL
    
    for (k in 2:(length(target_seq)-k)) {
      
      middle_site <- paste(target_seq[k:(k+ncol(pwm)-1)],collapse='')
      middle_site_vec <- c(middle_site_vec, middle_site)
      
    }
    
    candidate_sites <- NULL
    candidate_sites <- c(start_site_vec,middle_site_vec,rev(end_site_vec))
    
    # Iterate through all of the candidate sites and score with the MSS
    
    mss_vec <- NULL
    
    for (k in 1:length(candidate_sites)) {
      
      mss <- mss_score(candidate_sites[k],pwm)
      mss_vec <- c(mss_vec,mss)
      
    }
    
    max_mss <- round(as.numeric(max(mss_vec)),3)
    max_mss_vec <- c(max_mss_vec, max_mss)
  }
  
  }  
    
  # Use a lenient 95% threshold (usually about 0.7)
  
  thresh <- quantile(max_mss_vec,probs=seq(0,1,0.01))[96]
  
  rand_max_mss_vec <- max_mss_vec
  
  
  
  
  ########### Real data
  
  ## Now, iterate across target sequnces
  
  
  
  for (j in 1:length(target_seqs)) {
    max_mss_vec <- NULL
    top_site_vec <- NULL
    pos_vec <- NULL

    name_vec = NULL
    id_vec   = NULL
    start_site_vec <- NULL
    end_site_vec <- NULL
  
    target_seq <- target_seqs[[j]]
    
    name <- names(fasta)[j]
    #print(name)
    
    # Generate the start and end sites
    
    for (k in 1:ncol(pwm)) {
      
      start_site <- paste(paste(rep('_',ncol(pwm)-k),collapse=''),paste(target_seq[1:k],collapse=''),sep='')
      start_site_vec <- c(start_site_vec, start_site)
      
      end_site <- paste(paste(target_seq[(length(target_seq)-k+1):length(target_seq)],collapse=''),paste(rep('_',ncol(pwm)-k),collapse=''),sep='')
      end_site_vec <- c(end_site_vec, end_site)  
      
    }
    
    # Now extract all possile binding sites from the sequence
    
    middle_site_vec <- NULL
    
    for (k in 2:(length(target_seq)-k)) {
      
      middle_site <- paste(target_seq[k:(k+ncol(pwm)-1)],collapse='')
      middle_site_vec <- c(middle_site_vec, middle_site)
      
    }
    
    candidate_sites <- NULL
    candidate_sites <- c(start_site_vec,middle_site_vec,rev(end_site_vec))
    
    # Iterate through all of the candidate sites and score with the MSS
    
    mss_vec <- NULL
    
    for (k in 1:length(candidate_sites)) {
      
      mss <- mss_score(candidate_sites[k],pwm)
      mss_vec <- c(mss_vec,mss)
      
    }
    
    # We either report the maximum MSS or multiple MSS if there are several MSS above the threshold
    
    if (max(mss_vec) <= thresh) {
    
      pos_end <- which(mss_vec==max(mss_vec))
      pos_start <- pos_end-ncol(pwm)+1
      pos <- paste(pos_start,'-',pos_end,sep='')
      pos_vec <- c(pos_vec, pos)
      
      top_site <- candidate_sites[which(mss_vec==max(mss_vec))]
      top_site_vec <- c(top_site_vec,top_site)
      
      max_mss <- round(mss_vec[mss_vec==max(mss_vec)],3)
      max_mss_vec <- c(max_mss_vec, max_mss)
        

      
    } else {
      
      pos_end <- which(mss_vec>=thresh)
      pos_start <- pos_end-ncol(pwm)+1
      pos <- paste(pos_start,'-',pos_end,sep='')
      pos_vec <- c(pos_vec, pos)
      
      top_site <- candidate_sites[which(mss_vec>=thresh)]
      top_site_vec <- c(top_site_vec,top_site)
      
      max_mss <- mss_vec[which(mss_vec>=thresh)]
      max_mss_vec <- c(max_mss_vec, max_mss)  

      
    }  

    name_vec <- c(name_vec, rep(name,length(pos_vec)))
    id_vec <- c(id_vec, rep(id,length(pos_vec)))

      
    if(length(pos_vec) != length(max_mss_vec)) {stop()}
    if(length(pos_vec) != length(id_vec)) {print(pos_vec)
                                           print(id_vec)
                                           stop()}
    if(length(pos_vec) != length(name_vec)) {
        print(pos_vec)
        print(name_vec)
        stop()}
    
  
  
  max_mss_mat <- cbind(max_mss_vec,names(seq))

  temp_df <- data.frame(name_vec,pos_vec,id_vec,top_site_vec,max_mss_vec)
  colnames(temp_df) <- c('Gene_name','Start_End','PWM_name','PWM_hits','MSS')  

  SH3_PWM_df = rbind(SH3_PWM_df,temp_df)
}
}  
   
  file_name <- 'SH3_PWM_scan_HRR25Orthologs_MSS.csv'
  
  write_csv(SH3_PWM_df, file_name, col_names = TRUE)


ERROR: Error in sample(AAs, prob = sc_aa_freq, size = length(target_seq), replace = TRUE):  オブジェクト 'sc_aa_freq' がありません 


In [9]:
head(SH3_PWM_df)


Gene_name,Start_End,PWM_name,PWM_hits,MSS
<chr>,<chr>,<chr>,<chr>,<dbl>


In [180]:
print("Done")

[1] "Done"
